<a href="https://colab.research.google.com/github/fnunezs/geopandas/blob/main/convexhull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install descartes
!pip install mapclassify
!pip install folium 
!pip install fiona
!pip install utm

%matplotlib inline
import fiona
import folium
import mapclassify
import pandas as pd
import geopandas
import utm
from geopandas import GeoSeries
from shapely.geometry import Polygon, Point
import matplotlib.pyplot as plt

In [2]:
df_accidentes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/202105_accidentes_graves3.csv", sep=";")
#df_accidentes.head()

def utm_to_latlon(coords, zone_number, zone_letter):
  easting = coords[0]
  northing = coords[1]
  return list(reversed(utm.to_latlon(easting, northing, zone_number, zone_letter)))

df_accidentes["lat_lon"] = list(zip(df_accidentes["coordenada_x_utm"], df_accidentes["coordenada_y_utm"]))
df_accidentes["lat_lon"] = [utm_to_latlon(xy, 30, "N") for xy in df_accidentes.lat_lon]

geometry = [Point(xy) for xy in df_accidentes["lat_lon"]]
gdf_accidentes = geopandas.GeoDataFrame(df_accidentes, crs="EPSG:4326", geometry=geometry)

gdf_accidentes.set_crs(epsg=4326, inplace=True)

,num_expediente,fecha,hora,localizacion,numero,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,coordenada_x_utm,coordenada_y_utm,lat_lon,geometry
0,2021S007368,01/05/2021,17:40:00,"CALL. FUENCARRAL, 56",56,CENTRO,Atropello a persona,Despejado,Turismo,Conductor,De 60 a 64 años,Hombre,7.0,440570.53,4475032.91,"[-3.7005714798145273, 40.423806368762754]",POINT (-3.70057 40.42381)
1,2021S007368,01/05/2021,17:40:00,"CALL. FUENCARRAL, 56",56,CENTRO,Atropello a persona,Despejado,Turismo,Peatón,De 45 a 49 años,Mujer,7.0,440570.53,4475032.91,"[-3.7005714798145273, 40.423806368762754]",POINT (-3.70057 40.42381)
2,2021S007369,01/05/2021,17:50:00,"CALL. BLASCO DE GARAY, 62",62,CHAMBERÍ,Atropello a persona,Despejado,Furgoneta,Conductor,De 45 a 49 años,Hombre,14.0,439673.54,4476373.31,"[-3.7112718923750063, 40.435816722952076]",POINT (-3.71127 40.43582)
3,2021S007369,01/05/2021,17:50:00,"CALL. BLASCO DE GARAY, 62",62,CHAMBERÍ,Atropello a persona,Despejado,Furgoneta,Peatón,Menor de 5 años,Hombre,7.0,439673.54,4476373.31,"[-3.7112718923750063, 40.435816722952076]",POINT (-3.71127 40.43582)
4,2021S007369,01/05/2021,17:50:00,"CALL. BLASCO DE GARAY, 62",62,CHAMBERÍ,Atropello a persona,Despejado,Turismo,Conductor,Más de 74 años,Hombre,14.0,439673.54,4476373.31,"[-3.7112718923750063, 40.435816722952076]",POINT (-3.71127 40.43582)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2021S009509,31/05/2021,14:25:00,"CALL. SAN MATEO, 8",8,CENTRO,Atropello a persona,Despejado,Turismo,Peatón,Menor de 5 años,Hombre,1.0,440624.96,4475187.88,"[-3.6999443601105497, 40.42520629685752]",POINT (-3.69994 40.42521)
228,2021S009584,29/05/2021,15:40:00,"AVDA. RAFAELA YBARRA, 20",20,USERA,Atropello a persona,NaN,Turismo,Conductor,De 18 a 20 años,Mujer,NaN,439616.82,4470388.63,"[-3.7113726985892472, 40.38189980781438]",POINT (-3.71137 40.38190)
229,2021S009584,29/05/2021,15:40:00,"AVDA. RAFAELA YBARRA, 20",20,USERA,Atropello a persona,NaN,Turismo,Peatón,De 40 a 44 años,Hombre,NaN,439616.82,4470388.63,"[-3.7113726985892472, 40.38189980781438]",POINT (-3.71137 40.38190)
230,2021S009729,31/05/2021,8:46:00,AVDA. INGENIERO EMILIO HERRERA / CALL. INFANTA...,5,HORTALEZA,Atropello a persona,Despejado,Turismo,Conductor,De 25 a 29 años,Mujer,14.0,444549.26,4482429.27,"[-3.6543181721285936, 40.49071116391524]",POINT (-3.65432 40.49071)


In [3]:
# Creamos un mapa centrado en la ciudad de Madrid
mapa_accidentes = folium.Map(location=[40.429606584571246, -3.705951286477693], zoom_start=12, tiles='CartoDB positron')

# Añadimos al mapa los puntos donde se han producido accidentes
for _, r in gdf_accidentes.iterrows():   
  color = lambda x : "red" if (x > 2 and x < 5) else "orange"
  sim_geo = geopandas.GeoSeries(r['geometry'])
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(data=geo_j,
                         style_function=lambda x: {'fillColor': 'red'},
                         marker = folium.Marker(icon=folium.Icon(color=color(r['lesividad']), icon="car", prefix='fa'), size = (10,10)))
  descripcion = "Vehículo implicado: " + str(r['tipo_vehiculo']) + "\n" + "Edad victima: " + str(r['rango_edad']) 

  folium.Popup(descripcion).add_to(geo_j)
  geo_j.add_to(mapa_accidentes)

# Generamos convex-hull a partir de los puntos de los accidentes
sim_geo = gdf_accidentes['geometry'].unary_union.convex_hull

# Añadimos el polígono generado al mapa
convex_hull_accidentes = geopandas.GeoSeries([sim_geo]).to_json()
convex_hull_accidentes = folium.GeoJson(data=convex_hull_accidentes, 
                                 style_function=lambda x: {'fillColor': 'orange', 'color': 'white'})

convex_hull_accidentes.add_to(mapa_accidentes)

region_accidentes_graves = gdf_accidentes[gdf_accidentes.lesividad.between(3,4)]
# Generamos convex-hull a partir de los puntos de los accidentes
sim_geo_graves = region_accidentes_graves['geometry'].unary_union.convex_hull

# Añadimos el polígono generado al mapa
convex_hull_accidentes_graves = geopandas.GeoSeries([sim_geo_graves]).to_json()
convex_hull_accidentes_graves = folium.GeoJson(data=convex_hull_accidentes_graves, 
                                 style_function=lambda x: {'fillColor': 'red', 'color': 'black'})

mapa_accidentes


In [4]:
df_multas = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/202105_multas_trafico.csv", sep=";")
df_multas.head()

df_multas["lat_lon"] = list(zip(df_multas["COORDENADA-X"], df_multas["COORDENADA-Y "]))
df_multas["lat_lon"] = [utm_to_latlon(xy, 30, "N") for xy in df_multas.lat_lon]

geometry = [Point(xy) for xy in df_multas["lat_lon"]]
gdf_multas = geopandas.GeoDataFrame(df_multas, crs="EPSG:4326", geometry=geometry)

gdf_multas.set_crs(epsg=4326, inplace=True)

mapa_multas_accidentes = folium.Map(location=[40.429606584571246, -3.705951286477693], zoom_start=12, tiles='CartoDB positron')

# Generamos convex-hull a partir de los puntos de las multas
sim_geo = gdf_multas['geometry'].unary_union.convex_hull

convex_hull_multas = geopandas.GeoSeries([sim_geo]).to_json()
convex_hull_multas = folium.GeoJson(data=convex_hull_multas, 
                                 style_function=lambda x: {'fillColor': 'blue', 'color': 'blue'})


# Añadimos los polígonos al mapa
convex_hull_multas.add_to(mapa_multas_accidentes)
convex_hull_accidentes.add_to(mapa_multas_accidentes)
convex_hull_accidentes_graves.add_to(mapa_multas_accidentes)

mapa_multas_accidentes
